In [ ]:
from PyFastBDT import FastBDT
import basf2_mva
import modularAnalysis as ma
import ROOT as root
import numpy as np
import pandas as pd
import math
import uproot
import os
import sys
import importlib
import root_pandas as rp
import analysis_variables as a_v
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import metrics
from sklearn.model_selection import train_test_split
importlib.reload(a_v)

In [ ]:
BDT_type = 'contReweight'

sig_path = a_v.input_dir_long + '.parq'
bkg_path = a_v.input_dir_long + '.parq'

sig = pd.read_parquet(sig_path, engine='pyarrow').query('...>-0.15 and ...<0.1 and ...>0.5')
bkg = pd.read_parquet(bkg_path, engine='pyarrow').query('...>-0.15 and ...<0.1 and ...>0.5')

sig = sig.assign(Signal = [1] * len(sig.d0_Mbc))
bkg = bkg.assign(Signal = [0] * len(bkg.d0_Mbc))

np.random.seed(0)

if len(sig.d0_Mbc)>len(bkg.d0_Mbc):
    sig = sig.sample(frac=len(bkg.d0_Mbc)/len(sig.d0_Mbc), weights='...')
else:
    bkg = bkg.sample(frac=len(sig.d0_Mbc)/len(bkg.d0_Mbc), weights='...')
    
cr_vars = ['','','','....']

df = {}
sys = {}
inFilePath = {}

inFilePath['...'] = a_v.input_dir_long + '....parq'
inFilePath['...'] = a_v.input_dir_long + '....parq'
inFilePath['...'] = a_v.input_dir_long + '....parq'
inFilePath['...'] = a_v.input_dir_long + '....parq'
inFilePath['...'] = a_v.input_dir_long + '....parq'

for i in ['...','...','...','...',"..."]:
    df[i] = pd.read_parquet(inFilePath[i], engine='pyarrow')

target_var = ['Signal']
contRW_var = ['contReweight_new']

for i in range(1,21):
    if i % 10 == 0:
        print(f'{i}/20')
    np.random.seed(i)

    train_percent = 0.8
    test_percent = 1 - train_percent

    sig_y = sig.Signal
    sig_x = sig.drop('Signal',axis=1)
    bkg_y = bkg.Signal
    bkg_x = bkg.drop('Signal',axis=1)

    x = pd.concat([sig_x,bkg_x])
    y = pd.concat([sig_y,bkg_y])
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_percent)

    train = x_train.assign(Signal = y_train)
    
    x_train = train[cr_var]
    y_train = train[target_var]

    clf = FastBDT.Classifier(nTrees=200,depth=2,shrinkage=0.1) #best 200 2 0.1
    clf.fit(X=x_train, y=y_train)

    for j in ['...','...','...','...','...']:
        BDT_out = clf.predict(df[j][cr_var]).tolist()
        contRew = [ x/(1-x) for x in BDT_out]
        df[j][f'contReweight_new_{i}'] = contRew
        
        
for i in ['...','...','...','...','...']:
    df[i].to_root(inFilePath[i].replace(".parq","_SYS.root"),key=a_v.rootFilesTree)
    df[i].to_parquet(inFilePath[i].replace('.parq','_SYS.parq'), compression='GZIP')
